In [1]:
import requests as http
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from collections import Counter
import time
import calendar
import datetime
api_key = 'b333a19b2c0397e8e4c1224b49b3e7cd'
api_base = 'http://ws.audioscrobbler.com/2.0/?format=json&'
user_name = 'dude0faw3'
data = http.get(api_base + 'method=user.getrecenttracks&limit=200&api_key='\
                + api_key + '&user=' + user_name).json()

In [2]:
track_data = data['recenttracks']['track']
limit = data['recenttracks']['@attr']['totalPages']
page = 1
history = []
# for nested dictionary entry requests
class FakeNone:
    def get(*args):
        return FakeNone()
# ensures that there's no key error exception
def check_track(track):
    arr = [track.get('name', FakeNone()),
     track.get('artist', FakeNone()).get('#text', FakeNone()),
      track.get('date', FakeNone()).get('uts', FakeNone())
    ]
    return [None if type(x) == FakeNone else x for x in arr]

for track in track_data:
    history.append(check_track(track))
    
print('Starting! There are ' + limit + ' pages')
while int(data['recenttracks']['@attr']['page']) is not int(limit):
    if page % 25 is 0:
        print('Page %i' % page)
    page += 1
    data = http.get(api_base + 'method=user.getrecenttracks&limit=200&api_key=' + api_key + \
                    '&user=' + user_name + '&page=' + str(page)).json()
    for track in data['recenttracks']['track']:
        history.append(check_track(track))

df = pd.DataFrame(history)
df.columns = ['track_name', 'artist', 'date']
df.tail()

Starting! There are 155 pages
Page 25
Page 50
Page 75
Page 100
Page 125
Page 150


,track_name,artist,date
30997,This Is What It Feels Like,Armin van Buuren,1407180582
30998,Ecstasy,ATB,1407180320
30999,Spaceman - Drown The Fish Remix,Hardwell,1407179792
31000,Legacy,Eminem,1407179496
31001,Spaceman - Drown The Fish Remix,Hardwell,1407179059


## Goals:
- most listened to songs/ artists (as well as during a time)
- songs/artists you've neglected

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 31002 entries, 0 to 31001
Data columns (total 3 columns):
track_name    31002 non-null object
artist        31002 non-null object
date          30847 non-null object
dtypes: object(3)
memory usage: 726.7+ KB


In [20]:
# determine the n-most frequently played songs from a given pandas series
def determine_frequencies(vector_names, num_songs, neglected=False):
    if neglected:
        counter = Counter(vector_names)
        play_count = counter.most_common()
        index = len(play_count) - num_songs - 1
        return play_count[ index: ]
    else:
        counter = Counter(vector_names)
        return counter.most_common(num_songs)

determine_frequencies(df['track_name'], 10, neglected=True)

[('Out Of Control (feat. Ingrid)', 1),
 ('Stole The Show', 1),
 ('Starlight (Could You Be Mine) - Colin McLoughlin Remix', 1),
 ('Tremors (feat. Go Comet!)', 1),
 ('Stronger than I Was', 1),
 ('Starry Eyed', 1),
 ('Smile', 1),
 ('Tell Me', 1),
 ('Dystopia', 1),
 ('Care (ft. Madi)', 1),
 ('Numb (feat. Johnning)', 1)]

In [15]:
class TimeMachine:
    
    def __init__(self, year):
        self.seasons = {
            'spring': ( time.strptime('21 Mar ' + year, '%d %b %Y'),
                      time.strptime('21 Jun ' + year, '%d %b %Y')),
            'summer': ( time.strptime('21 Jun ' + year, '%d %b %Y'),
                      time.strptime('23 Sep ' + year, '%d %b %Y')),
            'fall': ( time.strptime('23 Sep ' + year, '%d %b %Y'),
                      time.strptime('21 Dec ' + year, '%d %b %Y')),
            'winter': ( time.strptime('21 Dec ' + year, '%d %b %Y'),
                        time.strptime('21 Mar ' + str(int(year) + 1), '%d %b %Y'))
        }
        self.year = year
        
    def set_year(self, new_year):
        self = self.__init__(new_year)
        
    def in_season(self, season, df):
        timeslice = self.seasons[season]
        def check_time(seconds):
            listen_time =  time.gmtime(int(seconds))
            return listen_time > timeslice[0] and listen_time < timeslice[1]
        arr_time_check = np.vectorize(check_time)
        return df[arr_time_check(df['date'])]
    
    def in_year(self, df):
        date = datetime.date(int(self.year), 1, 1)
        def check_time(seconds):
            listen_time =  datetime.date.fromtimestamp(int(seconds))
            return listen_time.year == date.year
        arr_time_check = np.vectorize(check_time)
        return df[ arr_time_check(df['date'])]
    

In [17]:
# make a timemachine to 2014, check out top artists from then
df = df.dropna()

obj = TimeMachine('2016')
winter_df = obj.in_season('winter', df)
determine_frequencies(winter_df['track_name'].values, 20)

[('Flash X (Said The Sky Remix)', 22),
 ('Slave II', 17),
 ('Same Old Song', 14),
 ('02:09', 14),
 ('Coming Down', 12),
 ('Uptown - Album Version (Edited)', 12),
 ('Been On', 12),
 ('Ride It', 11),
 ('A Word From Our Sponsor', 11),
 ('Fake Love', 10),
 ('Take U Out', 10),
 ('Drifting', 10),
 ('Some Kind Of Drug', 9),
 ('The Knowing', 9),
 ('Dum Dee Dum - JiKay Remix', 8),
 ('Loft Music - Album Version (Edited)', 8),
 ('American Idiot', 7),
 ('Kknow Me', 7),
 ('Next', 7),
 ("Can't Get Enough - Radio Edit", 7)]

In [21]:
df

,track_name,artist,date
1,Magic (feat. Rivers Cuomo),B.o.B,1484865465
2,Levels (Skrillex Remix),Avicii,1484865186
3,Ride It,Borgeous,1484864965
4,Many Ways - Radio Edit,Ferry Corsten feat. Jenny Wahlstrom,1484861133
5,Lights Down Low,Bei Maejor,1484860949
6,Frame of Mind,Tristam,1484860678
7,Acrylics,TNGHT,1484860423
8,Ride It,Borgeous,1484860202
9,International Love,Pitbull,1484843001
10,Wherever You Will Go,The Calling,1484842793
